TensorFlow Lite Conversion
---
Convert the full Keras model into a smaller TensorFlow Lite model file. Then, read in the raw hex bytes from the model file and write them to a separate C header file as an array.

In [28]:
from os.path import join
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import models
from scipy import stats
import c_writer

In [29]:
# Print versions
!python --version
print('Numpy ' + np.__version__)
print('TensorFlow ' + tf.__version__)
print('Keras ' + tf.keras.__version__)

Python 3.9.13
Numpy 1.21.5
TensorFlow 2.10.1
Keras 2.10.0


In [30]:
# Settings
models_path = 'models'  # Where we can find the model files (relative path location)
keras_model_name = 'LSTM_model'           # Will be given .h5 suffix
tflite_model_name = 'LSTM_model'          # Will be given .tflite suffix
c_model_name = 'fan_low_model'               # Will be given .h suffix1

In [31]:
# Load model
model = models.load_model(join(models_path, keras_model_name) + '.h5')

In [32]:
# Convert Keras model to a tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter._experimental_lower_tensor_list_ops = False
tflite_model = converter.convert()
open(join(models_path, tflite_model_name) + '.tflite', 'wb').write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\fede-\AppData\Local\Temp\tmpvkxig860\assets


INFO:tensorflow:Assets written to: C:\Users\fede-\AppData\Local\Temp\tmpvkxig860\assets


52744

In [33]:
# Construct header file
hex_array = [format(val, '#04x') for val in tflite_model]
c_model = c_writer.create_array(np.array(hex_array), 'unsigned char', c_model_name)
header_str = c_writer.create_header(c_model, c_model_name)

In [34]:
# Save C header file
with open(join(models_path, c_model_name) + '.h', 'w') as file:
    file.write(header_str)

Test Inference
---
Get known good values from the model for normal and anomaly samples to compare against C++ implementation.

In [39]:
# Saved Numpy test samples file location
sample_file_path = '..\\test_samples'
sample_file_name = 'normal_anomaly_samples'  # Will be given .npz suffix

max_measurements = 128
time_steps = 32

In [40]:
# Load test samples
with np.load(join(sample_file_path, sample_file_name) + '.npz') as data:
    normal_sample = data['normal_sample']
    anomaly_sample = data['anomaly_sample']
print(normal_sample.shape)
print(anomaly_sample.shape)
print(normal_sample[:5])

(200, 3)
(200, 3)
[[0.056425 0.041053 0.992775]
 [0.055632 0.042395 0.989115]
 [0.057096 0.041175 0.990152]
 [0.05673  0.041175 0.988749]
 [0.053436 0.042212 0.991677]]


In [44]:
# Test extracting features (median absolute deviation) using SciPy
import math


sample = normal_sample[0:max_measurements]                  # Truncate to 128 measurements
normal_x = []
for i in range(0, math.floor((max_measurements/time_steps)), time_steps):
            normal_x.append(sample[i*time_steps:((i+1)*time_steps)])
normal_x = np.array(normal_x)
sample = anomaly_sample[0:max_measurements]
anomaly_x = []
for i in range(0, math.floor((max_measurements/time_steps)), time_steps):
            anomaly_x.append(sample[i*time_steps:((i+1)*time_steps)])
anomaly_x = np.array(anomaly_x)
print("Normal MAD:", normal_x)
print("Anomaly MAD:", anomaly_x)

Normal MAD: [[[0.056425 0.041053 0.992775]
  [0.055632 0.042395 0.989115]
  [0.057096 0.041175 0.990152]
  [0.05673  0.041175 0.988749]
  [0.053436 0.042212 0.991677]
  [0.054961 0.042578 0.991006]
  [0.05551  0.038918 0.988688]
  [0.055327 0.042029 0.991189]
  [0.056974 0.041968 0.989542]
  [0.055815 0.041175 0.989786]
  [0.054473 0.039589 0.992165]
  [0.057157 0.04087  0.988932]
  [0.055693 0.040382 0.991372]
  [0.05673  0.039955 0.991555]
  [0.054351 0.042456 0.992287]
  [0.053985 0.041175 0.987773]
  [0.054778 0.040199 0.988993]
  [0.056059 0.042212 0.989969]
  [0.056974 0.041663 0.990152]
  [0.056181 0.038125 0.990762]
  [0.055815 0.04209  0.990701]
  [0.054412 0.040504 0.990579]
  [0.055937 0.042761 0.990579]
  [0.056181 0.042517 0.989725]
  [0.055205 0.040626 0.989359]
  [0.055449 0.041114 0.993202]
  [0.054534 0.042212 0.992104]
  [0.057218 0.040626 0.990457]
  [0.055998 0.039894 0.991555]
  [0.05551  0.0427   0.9882  ]
  [0.056181 0.04087  0.993141]
  [0.055693 0.041846 0.9908

In [45]:
# Perform inference and find MSE with normal sample
input_tensor = normal_x
pred = model.predict(input_tensor)
mse = np.mean(np.power(normal_x - pred, 2), axis=1)
print("Prediction:", pred)
print("MSE:", *mse)

1/1 [==============================] - 9s 9s/step
Prediction: [[[0.05315357 0.03911965 0.98851204]
  [0.05678287 0.04950383 0.9879505 ]
  [0.05822475 0.03983043 0.99268156]
  [0.05566148 0.03632066 0.9950259 ]
  [0.05403024 0.0378007  0.9903953 ]
  [0.05311585 0.04006122 0.98755264]
  [0.05306461 0.04202757 0.9869808 ]
  [0.05347701 0.04346972 0.9874937 ]
  [0.05399675 0.04433522 0.98835766]
  [0.05445153 0.04471284 0.9892086 ]
  [0.05478448 0.04473676 0.98989034]
  [0.05499605 0.04452975 0.99036634]
  [0.05510994 0.04418872 0.99065703]
  [0.05515468 0.04378313 0.99080104]
  [0.0551552  0.04335988 0.99083954]
  [0.05513033 0.04294851 0.99080706]
  [0.05509345 0.04256582 0.9907313 ]
  [0.05505273 0.04222036 0.99063236]
  [0.0550132  0.04191496 0.99052423]
  [0.0549775  0.04164921 0.99041575]
  [0.05494663 0.04142071 0.9903128 ]
  [0.05492072 0.04122604 0.9902185 ]
  [0.05489947 0.04106143 0.99013424]
  [0.05488229 0.04092305 0.99006027]
  [0.05486855 0.04080728 0.98999655]
  [0.05485781

In [47]:
# Perform inference and find MSE with anomaly sample
input_tensor = anomaly_x
pred = model.predict(input_tensor)
mse = np.mean(np.power(anomaly_x - pred, 2), axis=1)
print("Prediction:", pred)
print("MSE:", *mse)

1/1 [==============================] - 0s 120ms/step
Prediction: [[[0.05000145 0.05726043 0.76680046]
  [0.05928046 0.07657729 0.8790944 ]
  [0.06351883 0.08029208 0.9025014 ]
  [0.06272297 0.08264497 0.9117995 ]
  [0.06219133 0.08572852 0.9126205 ]
  [0.06228836 0.08846194 0.91228735]
  [0.06302409 0.09041958 0.9127359 ]
  [0.06410956 0.09166875 0.913786  ]
  [0.06525359 0.09235559 0.91500354]
  [0.0662854  0.09262889 0.9161187 ]
  [0.0671367  0.09262232 0.91701496]
  [0.06779973 0.09244157 0.9176713 ]
  [0.06829607 0.09216369 0.91811526]
  [0.06865747 0.09184176 0.91839075]
  [0.06891564 0.09151054 0.9185417 ]
  [0.06909783 0.09119137 0.9186068 ]
  [0.0692254  0.09089644 0.9186155 ]
  [0.06931476 0.09063151 0.9185897 ]
  [0.06937754 0.09039845 0.9185452 ]
  [0.06942211 0.09019656 0.91849196]
  [0.06945413 0.09002376 0.9184368 ]
  [0.06947757 0.08987715 0.9183839 ]
  [0.0694951  0.08975377 0.91833526]
  [0.0695084  0.08965056 0.9182919 ]
  [0.06951873 0.08956456 0.9182544 ]
  [0.06952